This notebook is largely based on the scRepertoire tutorial (https://www.borch.dev/uploads/screpertoire/articles/loading) and uses scRepertoire to analyse immune cell receptor data.


# Setup

**Only run this setup section on google colab, do not run this code if you use this notebook in a different environment.**

In [1]:
## https://stackoverflow.com/questions/70025153/how-to-access-the-shell-in-google-colab-when-running-the-r-kernel
shell_call <- function(command, ...) {
  result <- system(command, intern = TRUE, ...)
  cat(paste0(result, collapse = "\n"))
}

loadPackages = function(pkgs){
  myrequire = function(...){
    suppressWarnings(suppressMessages(suppressPackageStartupMessages(require(...))))
  }
  ok = sapply(pkgs, require, character.only=TRUE, quietly=TRUE)
  if (!all(ok)){
    message("There are missing packages: ", paste(pkgs[!ok], collapse=", "))
  }
}

## Setup R2U
download.file("https://github.com/eddelbuettel/r2u/raw/master/inst/scripts/add_cranapt_jammy.sh",
              "add_cranapt_jammy.sh")
Sys.chmod("add_cranapt_jammy.sh", "0755")
shell_call("./add_cranapt_jammy.sh")
bspm::enable()
options(bspm.version.check=FALSE)
shell_call("rm add_cranapt_jammy.sh")

46 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists...
Building dependency tree...
Reading state information...
ca-certificates is already the newest version (20230311ubuntu0.22.04.1).
gnupg is already the newest version (2.2.27-3ubuntu2.1).
gnupg set to manually installed.
wget is already the newest version (1.21.2-2ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.
-----BEGIN PGP PUBLIC KEY BLOCK-----

...
I5rGSOk715L9
=VuA5
-----END PGP PUBLIC KEY BLOCK-----
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit

Tracing function "install.packages" in package "utils"



In [2]:
## Install the R packages
cranPkgs2Install = c("BiocManager")
install.packages(cranPkgs2Install, ask=FALSE, update=TRUE, quietly=TRUE)
system("sudo apt install libgsl-dev")
BiocManager::install("scRepertoire")
install.packages('Seurat')
BiocManager::install("scater")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.19 (BiocManager 1.30.23), R 4.4.1 (2024-06-14)

Installing package(s) 'BiocVersion', 'scRepertoire'

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.rstudio.com

Bioconductor version 3.19 (BiocManager 1.30.23), R 4.4.1 (2024-06-14)

Installing package(s) 'scater'



In [3]:
## To simplify package loading, we created the loadPackages()
## function. But, if you don't have the function, you should
## use 'library(name_of_package)'
loadPackages = function(pkgs){
  myrequire = function(...){
    suppressWarnings(suppressMessages(suppressPackageStartupMessages(require(...))))
  }
  ok = sapply(pkgs, require, character.only=TRUE, quietly=TRUE)
  if (!all(ok)){
    message("There are missing packages: ", paste(pkgs[!ok], collapse=", "))
  }
}

pkgs = c("Seurat", "tidyverse", "scRepertoire", "scater")
loadPackages(pkgs)

‘SeuratObject’ was built under R 4.4.0 but the current version is
4.4.1; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘matrixStats’


The following object is masked from ‘package:dplyr’:

    count



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

   

In [4]:
# Download this file to your computer:
# https://drive.usercontent.google.com/download?id=1_YuRraDyg8UgF3oasjF0-jgPnwox-B24&export=download&authuser=0"
# and then open the folder icon on the left and drag the file there.
# It will appear on the bottom left ("scRep_example_full.rds") with a circle next to it
# that indicates upload progress. The full upload will take a few minutes, wait until it is complete.

# Introduction

<img src="https://camo.githubusercontent.com/f37d3bd3ae082e6e4c9765361d3bf44a4912f909bd701782753484ea4dd1c12e/68747470733a2f2f7777772e626f7263682e6465762f75706c6f6164732f73637265706572746f6972652f7265666572656e63652f666967757265732f73637265706572746f6972655f6865782e706e67" alt="drawing" width="200"/>

Single-cell sequencing is an emerging technology in the field of immunology and oncology that allows researchers to couple RNA quantification and other modalities, like immune cell receptor profiling at the level of an individual cell. A number of workflows and software packages have been created to process and analyze single-cell transcriptomic data. These packages allow users to take the vast dimensionality of the data generated in single-cell-based experiments and distill the data into novel insights. Unlike the transcriptomic field, there is a lack of options for software that allow for single-cell immune receptor profiling. Enabling users to easily combine RNA and immune profiling, the [scRepertoire](https://www.borch.dev/uploads/screpertoire/) framework supports use of 10x, AIRR, BD, MiXCR, Omniscope, TRUST4, and WAT3R single-cell clonal formats and interaction with popular R-based single-cell data pipelines.

Our tutorial below was adapted from the scRepertoire vignettes.


# What data to load into scRepertoire?
scRepertoire functions using the filtered_contig_annotations.csv output from the 10x Genomics Cell Ranger. This file is located in the ./outs/ directory of the VDJ alignment folder. To generate a list of contigs to use for scRepertoire:

load the filtered_contig_annotations.csv for each of the samples.
make a list in the R environment.


```

S1 <- read.csv(".../Sample1/outs/filtered_contig_annotations.csv")
S2 <- read.csv(".../Sample2/outs/filtered_contig_annotations.csv")
S3 <- read.csv(".../Sample3/outs/filtered_contig_annotations.csv")
S4 <- read.csv(".../Sample4/outs/filtered_contig_annotations.csv")

contig_list <- list(S1, S2, S3, S4)


```

## Other alignment workflows
Beyond the default 10x Genomic Cell Ranger pipeline outputs, scRepertoire supports the following single-cell formats:

AIRR
BD Rhapsody Multiomic Immune Profiling
Immcantation
JSON-formatted contig data
MiXCR
Omniscope OS-T/OS-B
Parse Evercode TCR/BCR
TRUST4
WAT3R
loadContigs() can be given a directory where the sequencing experiments are located and it will recursively load and process the contig data based on the file names. Alternatively, loadContigs() can be given a list of data frames and process the contig data



```
#Directory example
contig.output <- c("~/Documents/MyExperiment")
contig.list <- loadContigs(input = contig.output,
                           format = "TRUST4")

#List of data frames example
S1 <- read.csv("~/Documents/MyExperiment/Sample1/outs/barcode_results.csv")
S2 <- read.csv("~/Documents/MyExperiment/Sample2/outs/barcode_results.csv")
S3 <- read.csv("~/Documents/MyExperiment/Sample3/outs/barcode_results.csv")
S4 <- read.csv("~/Documents/MyExperiment/Sample4/outs/barcode_results.csv")

contig_list <- list(S1, S2, S3, S4)
contig.list <- loadContigs(input = contig.output,
                           format = "WAT3R")
```



# Example TCR Data in scRepertoire

scRepertoire comes with a data set from T cells derived from four patients with acute respiratory distress to demonstrate the functionality of the R package. More information on the data set can be found in the corresponding manuscript. The samples consist of paired peripheral-blood (B) and bronchoalveolar lavage (L), effectively creating 8 distinct runs for T cell receptor (TCR) enrichment. We can preview the elements in the list by using the head function and looking at the first contig annotation.

The built-in example data is derived from the 10x Cell Ranger pipeline, so it is ready to go for downstream processing and analysis.

Check that the uploaded file is in the correct folder and appears here. Note: it will already appear during upload, but you have to wait for upload to complete to then load it.

In [5]:
list.files(path = ".")

[1] "sample_data"

In [6]:
 scRep_example <- readRDS("scRep_example_full.rds")

Warning message in gzfile(file, "rb"):
“cannot open compressed file 'scRep_example_full.rds', probable reason 'No such file or directory'”


ERROR: Error in gzfile(file, "rb"): cannot open the connection


In [ ]:
scRep_example

# Combining Contigs into Clones
There are varying definitions of clones or clones in the literature. For the purposes of scRepertoire, we will use clone and define this as the cells with shared/trackable complementarity-determining region 3 (CDR3) sequences. Within this definition, one might use amino acid (aa) sequences of one or both chains to define a clone. Alternatively, we could use nucleotide (nt) or the V(D)JC genes (genes) to define a clone. The latter genes would be a more permissive definition of “clones”, as multiple amino acid or nucleotide sequences can result from the same gene combination. Another option to define clone is the use of the V(D)JC and nucleotide sequence (strict). scRepertoire allows for the use of all these definitions of clones and allows for users to select both or individual chains to examine.

## combineTCR
The output of combineTCR() will be a list of contig data frames that will be reduced to the reads associated with a single cell barcode. It will also combine the multiple reads into clone calls by either the nucleotide sequence (CTnt), amino acid sequence (CTaa), the VDJC gene sequence (CTgene), or the combination of the nucleotide and gene sequence (CTstrict).



In [ ]:
combined.TCR <- combineTCR(contig_list,
                           samples = c("P17B", "P17L", "P18B", "P18L",
                            "P19B","P19L", "P20B", "P20L"),
                           removeNA = FALSE,
                           removeMulti = FALSE,
                           filterMulti = FALSE)

head(combined.TCR[[1]])

In [ ]:
tail(combined.TCR[[1]])

## combineBCR
combineBCR() is analogous to combineTCR() with 2 major changes:


1.  Each barcode can only have a maximum of 2 sequences, if greater exists, the 2 with the highest reads are selected;
2.  The strict definition of a clone is based on the normalized Levenshtein edit distance of CDR3 nucleotide sequences and V-gene usage. For more information on this approach, please see the respective citation. This definition allows for the grouping of BCRs derived from the same progenitor that have undergone mutation as part of somatic hypermutation and affinity
maturation.


In [ ]:
BCR.contigs <- read.csv("https://www.borch.dev/uploads/contigs/b_contigs.csv")
combined.BCR <- combineBCR(BCR.contigs,
                           samples = "P1",
                           threshold = 0.85) #the level of similarity in sequences to group together. Default is 0.85.

head(combined.BCR[[1]])

In [ ]:
tail(combined.BCR[[1]])

# (optional) subsetClones
Likewise, we can remove specific list elements after combineTCR() using the subsetClones() function. In order to subset, we need to identify the vector we would like to use for subsetting (name) and the variable values to subset (variables). Below, we isolate just the 2 sequencing results from P18L and P18B.

In [ ]:
subset1 <- subsetClones(combined.BCR,
                        name = "sample",
                        variables = c("P1"))

head(subset1[[1]])

# clonalQuant

The first function to explore the clones is clonalQuant() to return the total or relative numbers of unique clones.

**scale**


*   TRUE - relative percent of unique clones scaled by the total size of the clonal repertoire.
*   FALSE - Report the total number of unique clones (default).

**chain**


*   “both” for combined chain visualization
*   “TRA”, “TRB”, “TRD”, “TRG”, “IGH” or “IGL” to select single chain


In [ ]:
clonalQuant(combined.TCR,
            cloneCall="strict",
            chain = "both",
            scale = TRUE)


In [ ]:
clonalQuant(combined.BCR,
            cloneCall="strict",
            chain = "both",
            scale = TRUE)


Another option here is to be able to define the visualization by data classes. Here, we used the combineTCR() list to define the Type variable as part of the naming structure. We can use the group.by to specifically use a column in the data set to organize the visualization.



In [ ]:
#add the Type in which the samples were processed and sequenced.
combined.TCR <- addVariable(combined.TCR,
                            variable.name = "Type",
                            variables = rep(c("B", "L"), 4))

clonalQuant(combined.TCR, cloneCall = "gene", group.by = "Type", scale = TRUE)

# clonalLength
We can look at the length distribution of the CDR3 sequences by calling the lengtheContig() function. Importantly, unlike the other basic visualizations, the cloneCall can only be “nt” or “aa”. Due to the method of calling clones as outlined above, the length should reveal a multimodal curve, this is a product of using the NA for the unreturned chain sequence and multiple chains within a single barcode.

**chain**


*   “both” for combined chain visualization
*   “TRA”, “TRB”, “TRD”, “TRG”, “IGH” or “IGL” to select single chain






In [ ]:
clonalLength(combined.TCR,
             cloneCall="aa",
             chain = "both")

In [ ]:
clonalLength(combined.TCR,
             cloneCall="aa",
             chain = "TRA",
             scale = TRUE)


# clonalCompare
We can also look at clones between samples and changes in dynamics by using the clonalCompare() function.

In [ ]:
clonalCompare(combined.TCR,
                  top.clones = 10,
                  samples = c("P17B", "P17L"),
                  cloneCall="aa",
                  graph = "alluvial")

We can also choose to highlight specific clones

In [ ]:
clonalCompare(combined.TCR,
              top.clones = 10,
              highlight.clones = c("CVVSDNTGGFKTIF_CASSVRRERANTGELFF", "NA_CASSVRRERANTGELFF"),
              relabel.clones = TRUE,
              samples = c("P17B", "P17L"),
              cloneCall="aa",
              graph = "alluvial")

In [ ]:
# show only specific clones
clonalCompare(combined.TCR, clones = c("CVVSDNTGGFKTIF_CASSVRRERANTGELFF", "NA_CASSVRRERANTGELFF"),
    relabel.clones = TRUE, samples = c("P17B", "P17L"), cloneCall = "aa", graph = "alluvial")

# Visualizing Clonal Dynamics
## clonalHomeostasis
By examining the clonal space, we effectively look at the relative space occupied by clones at specific proportions. Another way to think about this would be to think of the total immune receptor sequencing run as a measuring cup. In this cup, we will fill liquids of different viscosity - or different numbers of clonal proportions. Clonal space homeostasis asks what percentage of the cup is filled by clones in distinct proportions (or liquids of different viscosity, to extend the analogy). The proportional cut points are set under the cloneSize variable in the function and can be adjusted.

In [ ]:
clonalHomeostasis(combined.TCR,
                  cloneCall = "gene",
                  cloneSize = c(Rare = 0.001, Small = 0.01, Medium = 0.1, Large = 0.3, Hyperexpanded =
    1))

In [ ]:
clonalHomeostasis(combined.BCR,
                  cloneCall = "gene",
                  cloneSize = c(Rare = 0.001, Small = 0.01, Medium = 0.1, Large = 0.3, Hyperexpanded =
    1))

# clonalProportion
Like clonal space homeostasis above, clonal proportion places clones into separate bins. The key difference is that instead of looking at the relative proportion of the clone to the total, the clonalProportion() function will rank the clones by total number and place them into bins.

The clonalSplit represents the ranking of clonotypes by copy or frequency of occurrence, meaning 1:10 are the top 10 clonotypes in each sample. The default bins are under the clonalSplit variable in the function and can be adjusted, but they are as follows at baseline.

**clonalSplit**

*   10
*   100
*   1000
*   10000
*   30000
*   100000


In [ ]:
clonalProportion(combined.TCR,
                 cloneCall = "gene")

In [ ]:
clonalProportion(combined.BCR,
                 cloneCall = "gene")

In [ ]:
clonalProportion(combined.TCR,
                 cloneCall = "nt",
                 clonalSplit = c(1, 5, 10, 100, 1000, 10000))

In [ ]:
clonalProportion(combined.BCR,
                 cloneCall = "nt",
                 clonalSplit = c(1, 5, 10, 100, 1000, 10000))

# Summarizing Repertoires
## percentAA
Quantify the proportion of amino acids along the cdr3 sequence with percentAA(). By default, the function will pad the sequences with NAs up to the maximum of aa.length. Sequences longer than aa.length will be removed before visualization (default aa.length = 20).



In [ ]:
percentAA(combined.TCR,
          chain = "TRB",
          aa.length = 20)

##positionalEntropy
We can also quantify the level of entropy/diversity across amino acid residues along the cdr3 sequence. positionalEntropy() combines the quantification by residue of percentAA() with the diversity calls in clonalDiversity().

**method**:

“shannon” - Shannon Diversity; “inv.simpson” - Inverse Simpson Diversity;
“norm.entropy” - Normalized Entropy


In [ ]:
positionalEntropy(combined.TCR,
                  chain = "TRB",
                  aa.length = 20)

In [ ]:
positionalEntropy(combined.BCR,
                  chain = "both",
                  aa.length = 20)

## vizGenes
A visualization of the relative usage of genes of the TCR or BCR, using vizGenes(). vizGenes() is more adaptable to allow for comparisons across chains, scaling, etc, compared to other methods in the package like percentGenes() and percentVJ()

In [ ]:
vizGenes(combined.TCR,
         x.axis = "TRBV",
         y.axis = NULL,
         plot = "barplot",
         scale = TRUE)

#Comparing Clonal Diversity and Overlap


## ClonalDiversity
Diversity can also be measured for samples or by other variables. Diversity metrics calculated, include: “shannon”, “inv.simpson”, “norm.entropy”, “gini.simpson”, “chao1”, and “ACE”. Please see the manual for more information on each metric and the underlying calculations.

Inherent in diversity calculations is a bias for increasing diversity with increasing repertoire size. clonalDiversity() will automatically downsample to the smallest repertoire size and perform bootstrapping to return the mean diversity estimates. If the output of diversity values are strange or minimally variable, it is likely due to a sample with small repertoire size.

In [ ]:
clonalDiversity(combined.TCR,
                cloneCall = "gene")
#As a default, clonalDiversity() will return all the metrics calculated - “shannon”,
#“inv.simpson”, “norm.entropy”, “gini.simpson”, “chao1”, and “ACE”. Selecting a single or a subset of these methods using the metrics parameter.
# e.g. add 'metrics = c("shannon", "ACE")',

In [ ]:
clonalDiversity(combined.BCR,
                cloneCall = "gene")

(optional) you can add patient information and group by patient instead

In [ ]:
combined.TCR <- addVariable(
  combined.TCR,
  variable.name = "Patient",
  variables = c("P17", "P17", "P18", "P18", "P19","P19", "P20", "P20"))

clonalDiversity(combined.TCR,
                cloneCall = "gene",
                group.by = "Patient")

## clonalOverlap
If you are interested in measures of similarity between the samples loaded into scRepertoire, using clonalOverlap() can assist in the visualization.

The underlying clonalOverlap() calculation varies by the method parameter, more information on the exact calculations are available in the manual.
**method**:


*   “overlap” - overlap coefficient
*   “morisita” - Morisita’s overlap index
*   “jaccard” - Jaccard index
*   “cosine” - cosine similarity
*   “raw” - exact number of overlapping clones



In [ ]:
clonalOverlap(combined.TCR,
              cloneCall = "strict",
              method = "morisita")

Leave cluster by edit distance for now:
https://www.borch.dev/uploads/screpertoire/articles/clonal_cluster

#Combining Clones and Single-Cell Objects


## combineExpression and calculating cloneSize
After processing the contig data into clones via combineBCR() or combineTCR(), we can add the clonal information to the single-cell object using combineExpression().

Part of combineExpression() is calculating the clonal frequency and proportion, placing each clone into groups called cloneSize. The default cloneSize argument uses the following bins: c(Rare = 1e-4, Small = 0.001, Medium = 0.01, Large = 0.1, Hyperexpanded = 1), which can be modified to include more/less bins or different names.

Clonal frequency and proportion is dependent on the repertoires being compared, which we can modify the calculation using the group.by parameter, such as grouping by the Patient variable from above. If group.by is not set, combineExpression() will calculate clonal frequency, proportion, and cloneSize as a function of individual sequencing runs. In addition, cloneSize can use the frequency of clones when proportion = FALSE.

In [ ]:
#Making a Single-Cell Experiment object
sce <- Seurat::as.SingleCellExperiment(scRep_example)

sce <- combineExpression(combined.TCR,
                         sce,
                         cloneCall="gene",
                         group.by = "sample",
                         proportion = TRUE)

#Define color palette
colorblind_vector <- hcl.colors(n=7, palette = "inferno", fixup = TRUE)

# Visualize UMAP colored by default cloneSize groupings
scater::plotUMAP(sce, colour_by = "cloneSize") +
    scale_color_manual(values=rev(colorblind_vector[c(1,3,5,7)]))

Alternatively, if we want cloneSize to be based on the frequency of the clone, we can set proportion = FALSE and we will need to change the cloneSize bins to integers. If we have not inspected our clone data, setting the upper limit of the clonal frequency might be difficult - combineExpression() will automatically adjust the upper limit to fit the distribution of the frequencies. To demonstrate this, check out the Seurat object below:

In [ ]:
scRep_example <- combineExpression(combined.TCR,
                                   scRep_example,
                                   cloneCall="gene",
                                   group.by = "sample",
                                   proportion = FALSE,
                                   cloneSize=c(Single=1, Small=5, Medium=20, Large=100, Hyperexpanded=500))

Seurat::DimPlot(scRep_example, group.by = "cloneSize") +
    scale_color_manual(values=rev(colorblind_vector[c(1,3,4,5,7)]))

## clonalOccupy
We can also look at the count of cells by cluster assigned into specific frequency ranges by using the clonalOccupy() function and selecting the x.axis to display cluster or other variables in the meta data of the single cell object.

In [ ]:
clonalOccupy(scRep_example,
             x.axis = "seurat_clusters",
             proportion = FALSE, #change to TRUE to look at proportions!
             label = TRUE) #change to FALSE to not label the bars

**Optional but feel free to explore more visualization on single cell data in here https://www.borch.dev/uploads/screpertoire/articles/sc_visualizations**

# Session information

In [ ]:
print(sessionInfo())